In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from bitcoin.db import get_all_data
import numpy as np

df = get_all_data()


2018-03-17 15:26:21,786 root         INFO     Load data from SQL.


time                0
low                 0
high                0
open                0
close               0
volume              0
tw_sentiment        0
tw_followers        0
reddit_sentiment    0
google_sentiment    0
predicted_price     0
dtype: int64

In [ ]:
df[df.isnull().any(axis=1)]

In [ ]:
df.describe()

In [ ]:
df.corr()

In [ ]:
plt.plot(df['close'], 'b')

In [ ]:
max = df['close'].count()
limit = int(0.1 * max)
plt.plot(df['close'][0:max - limit], 'g')
plt.plot(df['close'][-limit:], 'r')

In [ ]:
df.hist(figsize=(10,10))

In [ ]:
X_train, X_test, y_train, y_test, scaler_x, scaler_y = core.prepare_inputs_outputs(df)

In [ ]:
history = core.train()

In [ ]:
plt.plot(history.history['mean_absolute_error'], 'r')
plt.plot(history.history['mean_squared_error'], 'g')

In [ ]:
core.test_order_percent()